In [1]:
from tools.PB_solver import *
import bempp.api
bempp.api.PLOT_BACKEND = "gmsh"
from tools.mesh_converter import *

### Simulation for a point charge centered

In [3]:
sim = Molecule('C:\\Users\\ian\Desktop\\forces_calculation','sphere','point',3.0,external_grid='n')
sim.save_info('C:\\Users\\ian\Desktop\\forces_calculation','sphere','point',3.0)


C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


### Simulation for two point charge with a 10 A distance between them

In [5]:
dist = [10,0,0]
simu1 = Two_proteins('C:\\Users\\ian\Desktop\\forces_calculation','sphere','point',2.0,'sphere','point',2.0,dist,external_grid='n',ep_in=2.)
simu1.save_info('C:\\Users\\ian\Desktop\\forces_calculation','sphere','point',2.0,'sphere','point',2.0,dist)

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
